## Assignment: Student Management Database in PostgreSQL
Objective
In this exercise, you will connect to a PostgreSQL database you created, build the tables for a simple student management system, insert sample data, and perform different queries — including JOIN operations.

Part 1: Database Connection
Create a new database in PostgreSQL named student_management.

Write Python code (using psycopg2) to connect to your student_management database.

Example:

In [7]:
import psycopg2

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="student_management",
    user="postgres",
    password="habiba2004#",
    host="localhost",
    port="5433"
)

cur = conn.cursor()
print("Database connected successfully.")


Database connected successfully.


## Part 2: Create Tables

Create the following tables using SQL:

### `students`
- `student_id` (SERIAL PRIMARY KEY)  
- `name` (VARCHAR)  
- `email` (VARCHAR)  
- `phone` (VARCHAR)  

### `courses`
- `course_id` (SERIAL PRIMARY KEY)  
- `course_name` (VARCHAR)  
- `credits` (INT)  

### `enrollments`
- `enrollment_id` (SERIAL PRIMARY KEY)  
- `student_id` (INT REFERENCES students(student_id))  
- `course_id` (INT REFERENCES courses(course_id))  
- `grade` (VARCHAR)  


In [ ]:
# CREATE STUDENTS TABLE
cur.execute("CREATE TABLE students (student_id SERIAL PRIMARY KEY , name VARCHAR(20) , email VARCHAR(50) , phone VARCHAR(50)) ")
conn.commit()
print("students table created")


# CREATE COURSES TABLE
cur.execute("CREATE TABLE courses (course_id SERIAL PRIMARY KEY , course_name VARCHAR(20) , credits INT) ")
conn.commit()
print("courses table created")

# CREATE ENROLLMENTS TABLE
cur.execute("CREATE TABLE enrollments (enrollment_id SERIAL PRIMARY KEY ,student_id INT REFERENCES students(student_id) , course_id INT REFERENCES courses(course_id) , grade VARCHAR(20)) ")
conn.commit()
print("enrollments table created")


enrollments table created


## Part 3: Insert Sample Data
Insert at least 3 records into each table.

Example:

In [15]:
sql = "INSERT INTO students (name, email, phone) VALUES (%s,%s,%s)"
data = [
    ('Alice Johnson', 'alice@example.com', '1234567890'),
    ('Bob Smith', 'bob@example.com', '9876543210'),
    ('Charlie Brown', 'charlie@example.com', '5555555555')
]
cur.executemany(sql, data)
conn.commit()
print("Data inserted into students table.")

sql2 ="INSERT INTO courses (course_name, credits) VALUES(%s,%s)"
data2 =[
    ('Mathematics', 3),
    ('Computer Science', 4),
    ('History', 2)]
cur.executemany(sql2, data2)
conn.commit()
print("Data inserted into courses table.")

sql3 = "INSERT INTO enrollments (student_id, course_id, grade) VALUES (%s,%s,%s)"
data3 = [
    (1, 1, 'A'),
    (1, 2, 'B'),
    (2, 3, 'A'),
    (3, 2, 'C')]
cur.executemany(sql3, data3)
conn.commit()
print("Data inserted into enrollments table.")

Data inserted into students table.
Data inserted into courses table.
Data inserted into enrollments table.


## Part 4: Query Tasks

Write SQL queries to answer the following questions:

1. **List all students.**
   ```sql
   SELECT * FROM students;


2. Find students who have a grade of 'A'.
Hint: Use a WHERE clause in the enrollments table to filter by grade.

SELECT * 
FROM enrollments
WHERE grade = 'A';


In [16]:
cur.execute("SELECT * FROM enrollments WHERE grade = 'A'")
cur.fetchall()

[(1, 1, 1, 'A'), (3, 2, 3, 'A')]

3. Show all courses with their credit hours.
Hint: Select course name and credit hours from the courses table.


SELECT course_name, credit_hours 
FROM courses;

In [20]:
cur.execute("SELECT course_name,credits FROM courses")
cur.fetchall()

[('Mathematics', 3), ('Computer Science', 4), ('History', 2)]

4. Find the courses a specific student (e.g., 'Alice Johnson') is enrolled in.
Hint: Use JOIN between students, enrollments, and courses.


SELECT s.name, c.course_name
FROM students s
JOIN enrollments e ON s.student_id = e.student_id
JOIN courses c ON e.course_id = c.course_id
WHERE s.name = 'Alice Johnson';

In [21]:
cur.execute("SELECT s.name, c.course_name FROM students s JOIN enrollments e ON s.student_id = e.student_id JOIN courseS c ON e.course_id = c.course_id WHERE s.name = 'Alice Johnson'")
cur.fetchall()

[('Alice Johnson', 'Mathematics'), ('Alice Johnson', 'Computer Science')]

5. List each student along with the number of courses they are enrolled in.
Hint: Use JOIN, GROUP BY, and COUNT().


SELECT s.name, COUNT(e.course_id) AS total_courses
FROM students s
JOIN enrollments e ON s.student_id = e.student_id
GROUP BY s.name;

In [25]:
cur.execute("SELECT s.student_id, s.name, COUNT(e.course_id) AS total_courses FROM students s LEFT JOIN enrollments e ON s.student_id = e.student_id GROUP BY s.student_id, s.name ORDER BY total_courses DESC")
cur.fetchall()

[(1, 'Alice Johnson', 2), (2, 'Bob Smith', 1), (3, 'Charlie Brown', 1)]

6. Find students who are not enrolled in any course.
Hint: Use LEFT JOIN and check for NULL in the enrollments table.


SELECT s.name
FROM students s
LEFT JOIN enrollments e ON s.student_id = e.student_id
WHERE e.course_id IS NULL;


In [28]:
cur.execute("SELECT  s.name FROM students s LEFT JOIN enrollments e ON s.student_id = e.student_id WHERE e.course_id IS NULL")
cur.fetchall()

[]

7. Show all enrollments sorted by grade (highest to lowest).
Hint: Use ORDER BY with DESC.


SELECT * 
FROM enrollments
ORDER BY grade DESC;



In [29]:
cur.execute("SELECT * FROM enrollments ORDER BY grade DESC")
cur.fetchall()

[(4, 3, 2, 'C'), (2, 1, 2, 'B'), (1, 1, 1, 'A'), (3, 2, 3, 'A')]

Part 5: Bonus
Write a query to find the average grade per course (Convert grades into numeric values if you want to calculate an average).

Add a instructor table and modify your queries to include instructor names.



In [ ]:
cur.execute("CREATE TABLE instructors (instructor_id SERIAL PRIMARY KEY, name VARCHAR(50), email VARCHAR(50))")
conn.commit()
print("instructors table created")

instructors table created


In [35]:
sql4 = "INSERT INTO instructors (name ,email) VALUES (%s,%s)"
data4 = [
    ('Dr. Smith', 'smith@univ.edu'),
    ('Prof. Brown', 'brown@univ.edu'),
    ('Dr. Lee', 'lee@univ.edu')
]
cur.executemany(sql4, data4)
conn.commit()
print("Data inserted into instructors table.")

Data inserted into instructors table.


In [36]:
cur.execute("ALTER TABLE courses ADD COLUMN instructor_id INT REFERENCES instructors(instructor_id)")

cur.execute("UPDATE courses SET instructor_id = 1 WHERE course_id = 1")
cur.execute("UPDATE courses SET instructor_id = 2 WHERE course_id = 2")
cur.execute("UPDATE courses SET instructor_id = 3 WHERE course_id = 3")
conn.commit()

In [37]:
cur.execute("""SELECT c.course_name ,i.name AS instructor,ROUND(AVG(CASE e.grade
            WHEN 'A' THEN 4
            WHEN 'B' THEN 3
            WHEN 'C' THEN 2
            WHEN 'D' THEN 1
            WHEN 'F' THEN 0
        END), 2) AS avg_grade
        FROM courses c
JOIN instructors i ON c.instructor_id = i.instructor_id
JOIN enrollments e ON c.course_id = e.course_id
GROUP BY c.course_name, i.name
ORDER BY avg_grade DESC;""")
cur.fetchall()

[('Mathematics', 'Dr. Smith', Decimal('4.00')),
 ('History', 'Dr. Lee', Decimal('4.00')),
 ('Computer Science', 'Prof. Brown', Decimal('2.50'))]